In [3]:
import os
import csv
import pandas as pd
import regex as re
import xlwt
import glob
import statistics
from datetime import datetime
from statistics import mode
import collections
import itertools
from iteration_utilities import deepflatten
import json
import csv
from collections import Counter


id = 1
#int(os.getenv("SGE_TASK_ID", 1))
outputfilename = f"word_collecter_{id}.csv"
print(outputfilename)
# # INPUT FOLDER FILE PATH TO INPUT .TXT FILES #
#filedirect = f"/restricted/projectnb/trucks/William/Mleczko_and_Desmond/nzlud/municipal_codes_all/{id}/"


filedirect = "C:/Users/clint/Desktop/nzlud/municipal_codes_all/test/"
filenames = sorted(glob.glob(filedirect + "*.txt")) # finds all .txt files in the folder your code is saved in

#word list input
words_list = [
    "shall", "must", "may not", "limit", "cap", "allocate", "approved", 
    "maximum", "restrict", "pay", "require", "provide", "dedicate", "reserve", 
     "set aside", "permit", "approve", "fee", "review", 
    "conditional", "plan", "application", "grant", "certification", "hearing", "report",  "authorize", "rezoning", "access", "allow", "prohibit", "deny", "fine", "ban",
    "special", "council", "board", "meetings", "up to", "minimum", "at least"
    "not more than","safeguard","instance","submit","impose","subject to","regulate","evaluation" ,"assessment,", "consideration","order to","pursuant","unless","comply","conform","preserve","prevent"
]
def extract_filename(path):
    import os
    base_name = os.path.basename(path)
    return os.path.splitext(base_name)[0]
extracted_filenames = [extract_filename(path) for path in filenames]
extracted_filenames.insert(0, "Words")
#extracted_filenames
header = extracted_filenames

# Example data to populate the CSV
data = words_list

#script_dir = os.path.dirname(os.path.abspath(__file__))
#csv_filepath = os.path.join(script_dir, outputfilename)

# Create and write to the CSV file
with open(outputfilename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)  # Write the header
    for entry in data:
        writer.writerow([entry])
print(f"CSV file created at: {outputfilename}")

def string_standardize(strng):
    replacement_dict = {

        "review": ["reviews", "reviewed", "reviewing"],
        "comply": ["compliance", "complied", "complies"],
        "restrict": ["restricted", "restricting"],
        "consideration": ["considerations", "considered", "considering","consider"],
        "evaluation": ["evaluations","evaluated","evaluate"],
        "assesment": ["assessments"],
        "regulate": ["regulated"],
        "not more than": ["no more than"],
        "ban": ["banned", "bans"],
        "prohibit": ["prohibited", "prohibits"],
        'safeguard': ['safeguards', 'safeguarding'],
        'conditional': ['conditionally', 'conditioned', 'conditions'],
        'allow': ['allowed', 'allowing', 'allows','allowable'],
        'plan': ['planned', 'planning', 'plans'],
        'aaprove': ['approved', 'approval', 'approving'],
        'laboratory': ['laboratories'],
        'amendment': ['amendments'],
        'acre': ['acres','ac\.'],
        ' percent': ['%'],
        'single family': ['single-family', 'singlefamily', 'single-family'],
        'multi family': ['multi-family', 'multifamily', 'multiple-family', 'multiple family'],
        'building permit': ['building permits'],
        'use permit': ['use permits'],
        'zoning permit': ['zoning permits'],
        'special permit': ['special permits'],
        'improvement location permit': ['improvement location permits'],
        'variation': ['variations'],
        'limit': ['limits', 'limited'],
        'application': ['applications',"apply"],
        'annual': ['annually'],
        'and': ['&'],
        'front': ['frt.'],
        'year': ['yearly'],
        'allocate': ['allocation'],
        'cap': ['capped', 'caps'],
        #'construction': ['construct', 'constructed','construction'],
        'pay': ['payment'],
        'fee': ['fees'],
        'provide': ['provided', 'provision'],
        'authorize': ['authorized', 'authorizing'],
        'grant': ['granted', 'granting'],
        'approve': ['approved', 'approving', 'approval'],
        'require': ['required', 'requires', 'requirement', 'requirements', 'requiring'],
        'issue': ['issued', 'issuance'],
        'reserve': ['reserved', 'reservation'],
        'dedicate': ['dedicated'],
        'building height': ['bldg. hgt.'],
        'designate': ['designated'],
        'incorporate': ['incorporated'],
        'moratorium': ['moratoria'],
        'minimum': ['min\.'],
        'maximum': ['max\.'],
        'permit': ['permits'],
        'plan': ['plans'],
        'variance': ['variances'],
        'lot': ['lots'],
        'dwelling unit': ['d\.u\.'],
        'dwelling unit per acre': ['du\/acre', 'dus/acre'],
        'unit': ['units'],
        'square feet': ['s\.f\.', 's\.\sf\.', 'sq\sft',
                        's\.feet', 's\.\sfeet', 's\.\sfeet', 'sq\.feet',
                        'sq\.\sfeet', 'sq\sfeet'],
        'mobile home park': ['m.h. - park'],
        'mobile home subdivision': ['m.h. - subdivision'],
        'acreage': ['acreage', 'Acreage'],
        'in lieu': ['in-lieu'],
        'set aside': ['set-aside']
    }  # update as needed
    for item in replacement_dict:
        for value in sorted(replacement_dict[item]):
            re_string = r'%s' % value
            strng = re.sub(re_string, r'%s' % item, strng)
    return strng

def fnote_fix(string):
    nums = re.findall(numbers, string, flags=re.IGNORECASE)

    nums_pr = [n for n in nums if "," in n]

    for n, num in enumerate(nums_pr):
        newnum = num.split(",")
        for i, j in enumerate(newnum):
            if len(j) > 3:
                newnum[i] = j[:3]
                finnum = "".join(newnum)
                string = re.sub(str(num), str(finnum), string)

    return string

numbers = r"""(?x)          # Turn on free spacing mode
            (
              #^a(?=\s)|     # Here we match a at the start of string before  whitespace
              #[-]?[0-9]+[,.]?[0-9]*[\/][0-9]+[,.]?[0-9]*|  # new numbers
              (?<!-\d*\.*|\.|table\s\d*\.*\d*\.*\d*\.*)\b[0-9]+[,.]?[0-9]*|  # new numbers
              (?<!-\d*\.*|\.|table\s\d*\.*\d*\.*\d*\.*)\b\d*\.?\,?\\?\d+ # HERE we match one or more digits
              #\b            # Initial word boundary 
              #(?:
              #    one|two|three|four|five|six|seven|eight|nine|ten| 
              #    eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen| 
              #    eighteen|nineteen|twenty|thirty|forty|fifty|sixty|seventy|eighty| 
              #    ninety|hundred|thousand|half
              #)             # A list of alternatives
              #\b            # Trailing word boundary
              )"""


word_collecter_1.csv
CSV file created at: word_collecter_1.csv


In [4]:


counter=0
for loc_entry in filenames:
    with open(loc_entry, 'r', errors='replace') as file:
        lines = file.read()
        ## the following lines consists of a series of text pre-processing steps ##
        lines1 = re.sub(r'"\d+', '', lines)
        lines2 = re.sub(r'[^\x00-\x7f]', r'-', lines1)
        lines3 = ' '.join(lines2.split())
        lines4 = re.sub(r'\n', ' ', lines3)  # takes away all new line indicators and replaces with a space, fixed a problem where regex wasn't catching some words when ran in the loop
        lines5 = re.sub(r'\t', ' ', lines4)  # removes \t characters
        lines6 = re.sub(r'\\', '', lines5)
        lines7 = re.sub(r'http\S+', '', lines6)
        lines8 = re.sub(r'\d+\.\d{3,}\.*\d*\.*\d*|\d+\.\d+\.\d+|\+\-+|\-{3,}', '', lines7)
        lines9 = re.sub(r'\[\d+\]', '', lines8)
        lines10 = re.sub(r'=', ' ', lines9)
        lines11 = re.sub(r'\bsf\b', "square feet", lines10)
        lines12 = re.sub(
            r'\b(Sterling\sCodifiers\,\sInc\.|Article|Chapter|Section|SECTION|Sections|Subsection|Sec|Prior\scode|Code|Ordinance|Ord|Lots|through|pg|pgs|Part)\b\.*\s*(No\.)*\s*\d*\w*\:*\.*\-*\d*\-*\d*\,*\.*\s*(and)*\s*\d*\-*\d*\-*\d*\,*\s*(and)*\s*\d*\-*\d*\-*\d*',
            '', lines11)
        lines13 = re.sub(r'\bAmended\sby\sOrd\.\sNo\.\s\d+\,\s\d+\/d+\/\d+|Amended\s\d{4}\b|amended\s\d{1,2}\/\d{1,2}\/\d{1,4}|\d+\/\d+\/\d+', '', lines12)
        lines14 = re.sub(r'\d+\:\d+', '', lines13)
        lines15 = lines14.lower()
        lines16 = re.sub(r'\b(january|february|march|april|may|june|july|august|september|october|november|december)\b\s\d+\,*\s\d{4}', '', lines15)
        lines17 = re.sub(r'pg\.|pgs\.\s\d+\-\d+|page\s\d+', '', lines16)
        lines18 = re.sub(r'\bft\.|\bft\b', "feet ", lines17)
        lines19 = string_standardize(str(lines18))
        lines20 = fnote_fix(str(lines19))
        lines21 = re.sub(r'[",!?*\[\]]', '', lines20)
        lines22 = re.sub(r';', ' ', lines21)
        lines23 = re.sub(r'\/acre|unit\/acre|unit\/net\sacre|unit\/gross\sacre|du\/ac\b', "unit per acre", lines22)
        lines24 = re.sub(r'\d+\/\d+\/\d+\ssterling\scodifiers\sinc\.', '', lines23)
        text = lines24.replace("(", "").replace(")", "").replace(".", "").replace(":", "").replace("/", "")
        text = text.replace("-", "").replace("'", "").replace(",", "").replace('``', '').replace("''", "")
        text = text.replace("[", "").replace("]", "")


    word_counts = {}

    for word in words_list:
        # Use re.findall to find all occurrences of the word or phrase
        matches = re.findall(r'\b' + re.escape(word) + r'\b', text)
        word_counts[word] = len(matches)
        
    df = pd.DataFrame(list(word_counts.items()), columns=['Word', 'Count'])
    existing_df = pd.read_csv(outputfilename)
    counter+=1
    print(extracted_filenames[counter])
    existing_df[extracted_filenames[counter]] = df['Count']
    existing_df.to_csv(outputfilename, index=False)




CA_SanLuisObispo
WA_Redmond
